In [1]:
import os
import matplotlib.pyplot as plt 
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.optim as optim
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from PIL import Image

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.Lambda(lambda x: x.convert('RGB') if x.mode == 'RGBA' else x.convert('RGB')),  # converted rgba and l grayscale into rgb
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [3]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)  # 224x224 -> 224x224
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)  # 224x224 -> 224x224
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1) # 224x224 -> 224x224
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # Reduces size by half: 224x224 -> 112x112

        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Flatten the 112x112x128 output to 512
        self.fc2 = nn.Linear(512, num_classes)  # Output layer, number of classes (e.g., 2 for binary classification)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 224x224 -> 112x112
        x = self.pool(F.relu(self.conv2(x)))  # 112x112 -> 56x56
        x = self.pool(F.relu(self.conv3(x)))  # 56x56 -> 28x28

        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor to a vector

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

model = CNNModel(num_classes=10)  # i have 10 output classes based on you can see in class_to_idx
print(model)


CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


In [7]:
model_32=CNNModel(num_classes=10)
model_32.load_state_dict(torch.load("model_state_dict.pth"))
data = torch.load('dataset_info.pth')
class_to_idx = data['class_to_idx']

/tmp/ipykernel_7023/30775028.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_32.load_state_dict(torch.load("model_state_dict.pth"))
/tmp/ipykernel_7023/30775028.py

In [8]:
model_32.eval()

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [9]:
test_folder = '/home/shahzaib/ANNProject/custom test images'  
image_paths = [os.path.join(test_folder, fname) for fname in os.listdir(test_folder) if fname.endswith(('.jpg', '.jpeg', '.png'))] 

idx_to_class = {v: k for k, v in class_to_idx.items()}  

for image_path in image_paths:
   
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) 
    
    # Make the prediction
    with torch.no_grad():
        outputs = model_32(image)
        _, predicted_class = torch.max(outputs, 1)  
    predicted_label = idx_to_class[predicted_class.item()]
    
    print(f"Image: {os.path.basename(image_path)} | Predicted label: {predicted_label}")

Image: brain tumor mein.jpeg | Predicted label: Brain Tumor 'Meningioma'
Image: CT-CHEST-0002-1024x1012.jpg | Predicted label: Lung Cancer 'Adenocarcin oma'
Image: pneumonia.jpeg | Predicted label: Normal XRAY No Pneumonia
Image: 4c1383482027c97c445f418c9b74b3_big_gallery.jpg | Predicted label: No Lung Cancer in this CT Scan
Image: brain tumor melegent.jpg | Predicted label: Brain Tumor 'Meningioma'
Image: largecell carcinoma.jpeg | Predicted label: Lung Cancer 'Large Cell carcinoma'
Image: Original-Lungs-CT-scan-image.png | Predicted label: Lung Cancer 'Large Cell carcinoma'
Image: normal xray no p.jpeg | Predicted label: Normal XRAY No Pneumonia
Image: normal xray.jpeg | Predicted label: Normal XRAY No Pneumonia
Image: normal xray.jpg | Predicted label: Normal XRAY No Pneumonia
Image: no tumor.jpg | Predicted label: No Brain Tumor in MRI
Image: P1neumonia xray.jpeg | Predicted label: Normal XRAY No Pneumonia
